**Example script to conduct a simple analysis in GEE**

Marco D Visser

In [ ]:
## import earth engine
import ee

## Run this belwo the first time
## authenticate
#ee.Authenticate()

## initialize session
ee.Initialize()




**Global forest cover & global albedo map **

In [95]:
## Global forest cover

## this imports the country boundary feauture from the EE data library
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

## load tree cover data
treeCover2014 = ee.Image("UMD/hansen/global_forest_change_2014")

## Albedo
albedo = ee.ImageCollection("MODIS/006/MCD43A3").filterDate('2017-01-01', '2017-03-31').min()


In [100]:
## Import the folium library (= visualizing geospatial data)
import folium
#import geehydro


## New method to display the image tiles
def addEELayer(self,eeImageObject,visparam,name):
  mapId_dict = ee.Image(eeImageObject).getMapId(visparam)
  folium.raster_layers.TileLayer(
      tiles = mapId_dict["tile_fetcher"].url_format,
      attr='Map Data &copy; <a href="http://earthengine.google.com/">Google Earth Engine </a>',
      name = name,
      overlay=True,
      control=True
      ).add_to(self)

## Add EE method to folium
folium.Map.add_ee_layer=addEELayer

## create a folium map object
myMap = folium.Map(location=[27.9506, -82.4572], zoom_start=7)

## Add country boundary
myMap.add_ee_layer(eeImageObject=ee.Image().paint(countries, 0, 1), visparam={'palette': ['red']},name='Countries Boundary')
myMap.add_ee_layer(treeCover2014.clip(countries),{'bands': ['treecover2000'], 'palette': ['000000','0000FF']},'Forest 2014')

## Get a composite of all Sentinal 2 images within a date range that include my points of interest.
namesPoi = [["Florida"],["Panama"],["Netherlands"]]
poiArray = [[-82.4,27.9] ,[-79.85,9.15], [5.2,52.15] ]

for i in range(3):
  poi = ee.Geometry.Point(poiArray[i])
  image = ee.ImageCollection('COPERNICUS/S2').filterBounds(poi).filterDate('2017-01-01', '2017-04-01').min()
  nir = image.select('B8')
  red = image.select('B4')
  ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI Layer')
  ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
  myMap.add_ee_layer(ndvi, ndviParams, 'NDVI image' + str(namesPoi[i]))

## add the albedo
blackSkyAlbedo = albedo.select('Albedo_BSA_Band1');
blackSkyAlbedoVis = {'min': 0.0,  'max': 400};
myMap.add_ee_layer(blackSkyAlbedo, blackSkyAlbedoVis, 'Black-Sky Albedo');

# To see a google satellite view as a basemap
#myMap.setOptions('HYBRID')
#myMap.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

## layer control button
myMap.add_child(folium.LayerControl())
display(myMap)